#### Module imports

In [1]:
import torch
import numpy as np
from numpy import asarray
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import permute
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
import pickle

#### Loading Files

In [2]:
# load pre processed files
imgs_path = Path('/home/richard/data/Schiefer/preprocessed_224')
assert imgs_path.is_dir()
files = sorted(imgs_path.glob('*.png'))

randomizer = np.random.RandomState(seed=99833)
randomizer.shuffle(files)

assert len(files) == 291 # all files are found
print("First 10 files are: {}".format([x.name for x in files[:10]]))

First 10 files are: ['CFB184_ortho_rgb_0110_preprocessed.png', 'CFB184_ortho_rgb_0107_preprocessed.png', 'CFB184_ortho_rgb_0111_preprocessed.png', 'CFB184_ortho_rgb_0123_preprocessed.png', 'CFB184_ortho_rgb_0105_preprocessed.png', 'CFB184_ortho_rgb_0170_preprocessed.png', 'CFB184_ortho_rgb_0184_preprocessed.png', 'CFB184_ortho_rgb_0231_preprocessed.png', 'CFB184_ortho_rgb_0271_preprocessed.png', 'CFB184_ortho_rgb_0188_preprocessed.png']


#### Loading Images

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def alter_image(img_name):
    image = Image.open(img_name)
    image_tensor = normalize(to_tensor(image)).unsqueeze(0)
    image_tensor = image_tensor.reshape(1,3,224,224)
    return image_tensor

def load_images_as_tensors(paths):
    images = [alter_image(image) for image in paths]
    return images

In [4]:
image_tensors = load_images_as_tensors(files)
assert len(image_tensors) == 291

#### Feature Extraction

Load the ResNet152 model

In [ ]:
# Load the pretrained model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)# Use the model object to select the desired layer
layer = model._modules.get('avgpool')
# Set model to evaluation mode
model.eval()

In [7]:
def get_feature_vector(image_tensor):
    # a dict to store the activations
    activation = {}
    def getActivation(name):
        # the hook signature
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    # register forward hooks on the layers of choice
    h1 = model.avgpool.register_forward_hook(getActivation('avgpool'))

    # forward pass -- getting the outputs
    out = model(image_tensor)

    # detach the hooks
    h1.remove()
    
    feature = torch.squeeze(activation['avgpool'])
    feature = torch.unsqueeze(feature,dim=0)
    return feature

# check
feature = get_feature_vector(image_tensors[0])
print(feature.shape)

torch.Size([1, 2048])


Get all features of the folder

In [8]:
def concat_tensors(features):
    fc = torch.cat(features)
    fc = fc.cpu().detach().numpy()
    return fc

def get_features(image_tensors):
    features = [get_feature_vector(tensor) for tensor in image_tensors]
    features = concat_tensors(features)
    return features

fc = get_features(image_tensors)
print(fc.shape)

Save features

In [10]:
results = {'filename': files,
           'features': fc,
           'layer_name': 'fc'}

feature_dir = imgs_path.parent / 'features'
Path(feature_dir).mkdir(parents=True, exist_ok=True)
with open(feature_dir / 'ResNet152_feature_std_224.pickle', 'wb') as f:
    pickle.dump(results, f)